In [16]:
# BSNIP

import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/BSNIP/AngleBasis1.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
age = []
sex = []
race = []
sz = []
fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for subtask in basis:
    m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    a = demo['Age_cal'][sub]
    s = demo['sex'][sub] == 's1.0'
    r = demo['Race'][sub] == 'AA'
    z = demo['DXGROUP_2'][sub] == 'SZP'
    if demo['DXGROUP_2'][sub] not in ['SZP', 'NC']:
        continue
    age.append(a)
    sz.append(z)
    sex.append(s)
    race.append(r)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-unk_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
age = np.array(age).astype('int')
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')
sz = np.array(sz).astype('int')
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, sex, race, sz, age, fc]])
print(np.mean(1-sex))
print(np.mean(1-race))
print(np.mean(1-sz))
print(np.mean(age))

[(441, 1, 264), (441, 1, 264), (441,), (441,), (441,), (441,), (441, 34716)]
0.4580498866213152
0.6439909297052154
0.5487528344671202
36.54875283446712


In [17]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps

paps = []
pres = []

for i in range(0,fc.shape[0],500):
    ps = tops(thetas[i:i+500], jitter[i:i+500])
    aps = np.mean(ps, axis=1)
    res = fc[i:i+500] - aps
    paps.append(aps)
    pres.append(res)
    print(f'Done {i}')
    
aps = np.concatenate(paps)
res = np.concatenate(pres)

print(ps.shape)
print(aps.shape)
print(res.shape)

Done 0
(441, 1, 34716)
(441, 34716)
(441, 34716)


In [18]:
aps1 = aps

print('Done')

Done


In [21]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

def tonp(x):
    return x.detach().cpu().numpy()

def totorch(x):
    return torch.from_numpy(x).float().cuda()

def totorchidcs(x):
    return torch.from_numpy(x).long().cuda() #F.one_hot(torch.from_numpy(x)).float().cuda()

def rmse(yt, yhat):
    return torch.mean((yt-yhat)**2)**0.5

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(34716,100).float().cuda()
        self.l2 = nn.Linear(100,2).float().cuda()
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x
    
def fit_mlp(xtr, ytr, verbose=False):
    xtr = totorch(xtr)
    ytr = totorchidcs(ytr)
    
    ce = nn.CrossEntropyLoss()
    mlp = MLP()
    optim = torch.optim.Adam(mlp.parameters(), lr=2e-4, weight_decay=2e-4)

    nepochs = 2000
    pperiod = 100

    for epoch in range(nepochs):
        optim.zero_grad()
        yhat = mlp(xtr)
        loss = ce(yhat, ytr)
        loss.backward()
        optim.step()
        if verbose:
            if epoch % pperiod == 0 or epoch == nepochs-1:
                print(f'{epoch} {float(loss)}')

    if verbose:
        print('Complete')
    
    return mlp
    
def predict_help(model, xt):
    xt = totorch(xt)
    with torch.no_grad():
        yhat = model(xt)
        return tonp(yhat)
    
print('Done')

Done


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import roc_auc_score

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def predict(xtr, xt, ytr, yt, lst):
    reg = fit_mlp(xtr, ytr, verbose=False)
    p = predict_help(reg, xt)
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)
    return p
    
def get_res(fctr, fct, abtr, abt):
    return fctr-abtr, fct-abt

def combine(yt, p0, p1, lst):
    p = (p0+p1)/2
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)

rfc = []

rab1 = []
rab3 = []
rab5 = []
rab10 = []
rab15 = []
rab20 = []

rres1 = []
rres3 = []
rres5 = []
rres10 = []
rres15 = []
rres20 = []

rens1 = []
rens3 = []
rens5 = []
rens10 = []
rens15 = []
rens20 = []

rbest = []

for i in range(10):

    x0tr, x0t, x1tr, x1t, x2tr, x2t, x3tr, x3t, x4tr, x4t, x5tr, x5t, x6tr, x6t, ytr, yt = train_test_split(
        fc, aps1, aps3, aps5, aps10, aps15, aps20, sz, stratify=sz, train_size=0.8)
    
    x1atr, x1at = get_res(x0tr, x0t, x1tr, x1t)
    x2atr, x2at = get_res(x0tr, x0t, x2tr, x2t)
    x3atr, x3at = get_res(x0tr, x0t, x3tr, x3t)
    x4atr, x4at = get_res(x0tr, x0t, x4tr, x4t)
    x5atr, x5at = get_res(x0tr, x0t, x5tr, x5t)
    x6atr, x6at = get_res(x0tr, x0t, x6tr, x6t)

    predict(x0tr, x0t, ytr, yt, rfc)
    
    p1 = predict(x1tr, x1t, ytr, yt, rab1)
    p2 = predict(x2tr, x2t, ytr, yt, rab3)
    p3 = predict(x3tr, x3t, ytr, yt, rab5)
    p4 = predict(x4tr, x4t, ytr, yt, rab10)
    p5 = predict(x5tr, x5t, ytr, yt, rab15)
    p6 = predict(x6tr, x6t, ytr, yt, rab20)
    
    p1a = predict(x1atr, x1at, ytr, yt, rres1)
    p2a = predict(x2atr, x2at, ytr, yt, rres3)
    p3a = predict(x3atr, x3at, ytr, yt, rres5)
    p4a = predict(x4atr, x4at, ytr, yt, rres10)
    p5a = predict(x5atr, x5at, ytr, yt, rres15)
    p6a = predict(x6atr, x6at, ytr, yt, rres20)
    
    combine(yt, p1, p1a, rens1)
    combine(yt, p2, p2a, rens3)
    combine(yt, p3, p3a, rens5)
    combine(yt, p4, p4a, rens10)
    combine(yt, p5, p5a, rens15)
    combine(yt, p6, p6a, rens20)

    combine(yt, p6, p1a, rbest)
    
    print('---')
    
print(np.mean(rfc), np.std(rfc))
print(np.mean(rbest), np.std(rbest))

print(np.mean(rab1), np.std(rab1))
print(np.mean(rab3), np.std(rab3))
print(np.mean(rab5), np.std(rab5))
print(np.mean(rab10), np.std(rab10))
print(np.mean(rab15), np.std(rab15))
print(np.mean(rab20), np.std(rab20))

print(np.mean(rres1), np.std(rres1))
print(np.mean(rres3), np.std(rres3))
print(np.mean(rres5), np.std(rres5))
print(np.mean(rres10), np.std(rres10))
print(np.mean(rres15), np.std(rres15))
print(np.mean(rres20), np.std(rres20))

print(np.mean(rens1), np.std(rens1))
print(np.mean(rens3), np.std(rens3))
print(np.mean(rens5), np.std(rens5))
print(np.mean(rens10), np.std(rens10))
print(np.mean(rens15), np.std(rens15))
print(np.mean(rens20), np.std(rens20))

0.8959183673469387
0.8326530612244898
0.8806122448979592
0.8642857142857143
0.8857142857142857
0.888265306122449
0.8877551020408164
0.8255102040816327
0.801530612244898
0.7698979591836734
0.7117346938775511
0.6566326530612245
0.7316326530612244
0.8673469387755102
0.8943877551020408
0.8678571428571429
0.8806122448979592
0.8826530612244898
0.8775510204081632
0.8938775510204082
---
0.810204081632653
0.7540816326530612
0.7872448979591837
0.7806122448979592
0.7959183673469388
0.8096938775510204
0.7908163265306122
0.7520408163265306
0.7198979591836736
0.7591836734693879
0.745408163265306
0.7352040816326532
0.7510204081632653
0.8107142857142857
0.8290816326530612
0.8153061224489796
0.8244897959183674
0.8285714285714286
0.8209183673469388
0.810204081632653
---
0.8112244897959184
0.7234693877551021
0.801530612244898
0.7908163265306123
0.801530612244898
0.8081632653061225
0.8045918367346939
0.8311224489795918
0.7698979591836734
0.7357142857142859
0.7341836734693877
0.6948979591836735
0.712244897